![Funcional Health Tech](logo.png "Logo")

**COVID19 - Análise do Surto Brasileiro**

# Pré-Processamento de Dados

Neste *notebook* integramos e transformamos diversas fontes de dados, para que estejam disponíveis em análises posteriores. Empregaremos aqui as seguintes fontes:

  - [Our World in Data](https://ourworldindata.org/coronavirus-source-data)
  - [Banco Mundial](https://datacatalog.worldbank.org/dataset/world-development-indicators)
  

## Preliminares
Bibliotecas, configurações e parâmetros.

In [4]:
import os
import math
import hyperopt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 8]
import seaborn as sns
sns.set()

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [8]:
plt.style.use('fivethirtyeight')

In [40]:
os.getcwd()

In [41]:
os.chdir('covid19-analytics/notebooks')
os.getcwd()

## Leitura e Preparação de Dados

Vamos empregar dados do [Our World in Data](https://ourworldindata.org/coronavirus-source-data ) como nossa fonte principal de dados de COVID-19, mas também a enriqueceremos com outras informações.

### Dados de COVID-19

Dados de infecções e óbitos, além de algumas informações demográficas.

In [10]:
df_epidemy_data = pd.read_csv('../data/ourworldindata.org/coronavirus-source-data/full_data.csv', parse_dates=['date'])
df_epidemy_data.head(2)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0


In [11]:
df_locations_data = pd.read_csv('../data/ourworldindata.org/coronavirus-source-data/locations.csv')
df_locations_data.head(2)

,countriesAndTerritories,location,continent,population_year,population
0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0
1,Albania,Albania,Europe,2020.0,2877800.0


Vamos enriquecer os dados com informações adicionais sobre as localidades.

In [12]:
df_epidemy_data = df_epidemy_data.merge(df_locations_data, left_on='location', right_on='countriesAndTerritories', suffixes=('', '_demography'))

df_epidemy_data['total_cases_per_capita'] = df_epidemy_data['total_cases'] / df_epidemy_data['population']
df_epidemy_data['total_deaths_per_capita'] = df_epidemy_data['total_deaths'] / df_epidemy_data['population']

df_epidemy_data.head(2)

,date,location,new_cases,new_deaths,total_cases,total_deaths,countriesAndTerritories,location_demography,continent,population_year,population,total_cases_per_capita,total_deaths_per_capita
0,2019-12-31,Afghanistan,0,0,0,0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0,0.0,0.0
1,2020-01-01,Afghanistan,0,0,0,0,Afghanistan,Afghanistan,Asia,2020.0,38928341.0,0.0,0.0


Faixa de datas para a qual temos dados.

In [13]:
print(f"From {df_epidemy_data['date'].iloc[0].date()} to {df_epidemy_data['date'].iloc[-1].date()}")

From 2019-12-31 to 2020-04-19


Salvemos para uso posterior.

In [14]:
df_epidemy_data.to_csv('../data/preprocessed/covid_progression.csv', index=False)

### Banco Mundial: World Development Indicators

Incorporemos agora dados do Banco Mundial.

In [15]:
df_wdi = pd.read_csv('../data/large/datacatalog.worldbank.org/dataset/world-development-indicators/WDIData.csv')
df_wdi.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,82.783289,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,86.428272,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN


Vamos computar o valor mais recente disponível para os diversos indicadores.

In [16]:
def latest_not_nan(values):
    values = np.array(values)
    values = values[~np.isnan(values)]
    if len(values) > 0:
        return values[-1]
    else:
        return None
    
df_wdi['latest_value_available'] = df_wdi.apply(lambda row: latest_not_nan([row[str(y)] for y in range(2005, 2020)]), axis=1)
df_wdi.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64,latest_value_available
0,Arab World,ARB,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,83.120303,83.533457,83.897596,84.171599,84.510171,NaN,NaN,NaN,NaN,84.510171
1,Arab World,ARB,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,87.070576,88.176836,87.342739,89.130121,89.678685,90.273687,NaN,NaN,NaN,90.273687


Quantos indicadores totais temos?

In [17]:
df_wdi['Indicator Name'].nunique()

1429

Escolhamos então alguns indicadores, pois temos centenas deles. Como sugestão, podemos tomar o que o próprio Banco Mundial destaca para os países, por exemplo para o [Brasil](https://data.worldbank.org/country/brazil).

In [18]:
selected_world_bank_indicator_codes = ['NY.GDP.MKTP.CD', # GDP
                                       'SP.POP.TOTL', # Population
                                       'SE.PRM.ENRR', # School enrollment, primary (% gross)
                                       'SE.PRM.TENR',
                                       'EN.ATM.CO2E.PC', # CO2
                                       'SP.DYN.LE00.IN', # life expectancy
                                       'NY.GNP.PCAP.CD', # GNI per capita
                                       ]

In [19]:
df_wdi = df_wdi[df_wdi['Indicator Code'].isin(selected_world_bank_indicator_codes)]

Como fica para um país específico?

In [20]:
df_wdi[df_wdi['Country Name'] == 'Brazil']

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,Unnamed: 64,latest_value_available
104334,Brazil,BRA,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,NaN,NaN,NaN,NaN,NaN,NaN,...,9.854993e+01,9.732389e+01,9.743620e+01,9.685231e+01,9.720202e+01,9.755117e+01,NaN,NaN,NaN,9.755117e+01
104510,Brazil,BRA,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,6.498859e-01,6.621305e-01,7.017755e-01,7.060494e-01,6.998843e-01,6.764552e-01,...,2.358548e+00,2.505409e+00,2.612934e+00,NaN,NaN,NaN,NaN,NaN,NaN,2.612934e+00
104784,Brazil,BRA,GDP (current US$),NY.GDP.MKTP.CD,1.516557e+10,1.523685e+10,1.992629e+10,2.302148e+10,2.121189e+10,2.179004e+10,...,2.465189e+12,2.472806e+12,2.455994e+12,1.802214e+12,1.796275e+12,2.053595e+12,1.868626e+12,NaN,NaN,1.868626e+12
104818,Brazil,BRA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.236000e+04,1.281000e+04,1.210000e+04,1.016000e+04,8.930000e+03,8.670000e+03,9.140000e+03,NaN,NaN,9.140000e+03
105018,Brazil,BRA,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,5.414300e+01,5.463400e+01,5.513000e+01,5.562700e+01,5.612100e+01,5.661000e+01,...,7.420900e+01,7.448300e+01,7.474500e+01,7.499400e+01,7.523000e+01,7.545600e+01,7.567200e+01,NaN,NaN,7.567200e+01
105372,Brazil,BRA,"Population, total",SP.POP.TOTL,7.217923e+07,7.431134e+07,7.651433e+07,7.877266e+07,8.106457e+07,8.337353e+07,...,1.992873e+08,2.010359e+08,2.027637e+08,2.044718e+08,2.061631e+08,2.078338e+08,2.094693e+08,NaN,NaN,2.094693e+08
105505,Brazil,BRA,"School enrollment, primary (% gross)",SE.PRM.ENRR,NaN,NaN,NaN,NaN,NaN,NaN,...,1.345204e+02,1.139198e+02,1.142547e+02,1.127733e+02,1.139454e+02,1.154478e+02,NaN,NaN,NaN,1.154478e+02


Vamos deixar apenas algumas colunas.

In [21]:
df_wdi = df_wdi[['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code', 'latest_value_available']]
df_wdi.head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,latest_value_available
17,Arab World,ARB,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,85.384220
193,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,4.886988


In [22]:
df_wdi[df_wdi['Country Name'] == 'Brazil'].head()

,Country Name,Country Code,Indicator Name,Indicator Code,latest_value_available
104334,Brazil,BRA,"Adjusted net enrollment rate, primary (% of pr...",SE.PRM.TENR,9.755117e+01
104510,Brazil,BRA,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,2.612934e+00
104784,Brazil,BRA,GDP (current US$),NY.GDP.MKTP.CD,1.868626e+12
104818,Brazil,BRA,"GNI per capita, Atlas method (current US$)",NY.GNP.PCAP.CD,9.140000e+03
105018,Brazil,BRA,"Life expectancy at birth, total (years)",SP.DYN.LE00.IN,7.567200e+01


In [23]:
df_wdi_pivot = df_wdi.pivot(index='Country Name', columns='Indicator Code', values='latest_value_available')
df_wdi_pivot.head()

Indicator Code,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GNP.PCAP.CD,SE.PRM.ENRR,SE.PRM.TENR,SP.DYN.LE00.IN,SP.POP.TOTL
Country Name,,,,,,,
Afghanistan,0.293946,1.936297e+10,550.0,103.99616,NaN,64.486,37172386.0
Albania,1.978763,1.510250e+10,4860.0,106.99342,97.44221,78.458,2866376.0
Algeria,3.735520,1.737580e+11,3920.0,109.88282,99.64750,76.693,42228429.0
American Samoa,NaN,6.360000e+08,NaN,NaN,NaN,NaN,55465.0
Andorra,5.832906,3.236544e+09,NaN,NaN,NaN,NaN,77006.0


Salvemos os indicadores para cada país.

In [24]:
df_wdi_pivot.to_csv('../data/preprocessed/world_bank_selected_wdi.csv')

In [25]:
df_epidemy_data_enriched = df_epidemy_data.merge(df_wdi_pivot, left_on='location', right_on='Country Name')
#df_epidemy_data_enriched.head()

Salvemos também a versão dos dados epidêmicos enriquecida com os indicadores do Banco Mundial.

In [26]:
df_epidemy_data_enriched.to_csv('../data/preprocessed/covid_wdi-enriched_progression.csv', index=False)

### Estruturação para Estudos

Convém agora colocar os dados escolhidos em formatos mais adequados para as análises seguintes.

Primeiramente, separemos os casos detectados dos óbitos, tendo os países como colunas. Isso facilitará a comparação temporal absoluta dos diversos países.

In [27]:
df_total_cases = df_epidemy_data_enriched.pivot(index='date', columns='location', values='total_cases')
#df_total_cases.tail()

In [28]:
df_total_cases.to_csv('../data/preprocessed/covid_total_cases.csv')

In [29]:
df_total_deaths = df_epidemy_data.pivot(index='date', columns='location', values='total_deaths')
#df_total_deaths.tail()

In [30]:
df_total_deaths.to_csv('../data/preprocessed/covid_total_deaths.csv')

Precisamos também ajustar os dados para mostrarem o progresso a partir do primeiro caso detectado, de modo que a progressão relativa da epitemia também possa ser analisada. Para tanto, a seguinte função nos dá, para cada métrica escolhida (`'total_cases'`, `'total_cases_per_capita'`, `'total_cases'` e `'total_cases_per_capita'`) tanto o *dataframe* apropriado quanto um dicionário que mapeia, para cada país, quando foi o primeiro momento em que e métrica escolhida foi diferente de zero.

In [31]:
def build_dataframe_with_metric_per_epidemy_day(metric_name, base_metric_name='total_cases'):
    first_dates_per_location = {}
    progresses_per_epidemy_day = []
    
    locations = df_epidemy_data['location'].unique()
    
    for location in locations:
        
        # the first date is calculated from a common base metric, so that multiple metrics can be properly compared.
        first_date = df_epidemy_data[(df_epidemy_data['location'] == location) & \
                                      (df_epidemy_data[base_metric_name] != 0)]['date'].iloc[0]
        
        df_location = df_epidemy_data[df_epidemy_data['location'] == location]
        df_location = df_location[df_location['date'] >= first_date]
        s_selected_metric = df_location[metric_name].rename(location).reset_index(drop=True)
        progresses_per_epidemy_day.append(s_selected_metric)

        first_dates_per_location[location] = first_date
        
    df_metric_per_epidemy_day = pd.concat(progresses_per_epidemy_day, axis=1)
    return df_metric_per_epidemy_day, first_dates_per_location


In [32]:
df_total_cases_per_epidemy_day, first_cases_per_location = build_dataframe_with_metric_per_epidemy_day(metric_name='total_cases')
#df_total_cases_per_epidemy_day.head()

In [33]:
df_total_cases_per_epidemy_day.to_csv('../data/preprocessed/covid_total_cases_per_epidemy_day.csv')

In [34]:
df_total_cases_per_capita_per_epidemy_day, first_cases_per_location = \
                            build_dataframe_with_metric_per_epidemy_day(metric_name='total_cases_per_capita')

#df_total_cases_per_capita_per_epidemy_day.head()

In [35]:
df_total_cases_per_capita_per_epidemy_day.to_csv('../data/preprocessed/covid_total_cases_per_capita_per_epidemy_day.csv')

In [36]:
df_total_deaths_per_epidemy_day, first_deaths_per_location = \
                                    build_dataframe_with_metric_per_epidemy_day(metric_name='total_deaths')

#df_total_deaths_per_epidemy_day.head()

In [37]:
df_total_deaths_per_epidemy_day.to_csv('../data/preprocessed/covid_total_deaths_per_epidemy_day.csv')

In [38]:
df_total_deaths_per_capita_per_epidemy_day, first_deaths_per_location = \
                                    build_dataframe_with_metric_per_epidemy_day(metric_name='total_deaths_per_capita')

#df_total_deaths_per_capita_per_epidemy_day.head()

In [39]:
df_total_deaths_per_capita_per_epidemy_day.to_csv('../data/preprocessed/covid_total_deaths_per_capita_per_epidemy_day.csv')